# Homework 2

### Deadline: Friday 28 November 2025 (by 21h00)

### Credits: 20 points

### Instructions:

- The quiz is individual. Please include your name in the notebook.


- Within a **single python notebook (.ipynb)**, solve the following problems.

## Name: Mejia Perez Bady Fernando

# Supersonic turbulence: Calculus and Fourier analysis (20 points)

We want to study the properties of 2D (magnetohydrodynamical) turbulent flows using numerical calculus and Fourier analysis. As explained in class, in a turbulence simulation the gas is initially still and has a uniform density distribution. Then, turbulence is generated by appying a time-dependent, spatially-varying, stochastic force field. Such force field continuously injects mechanical energy into the 2D box and maintains supersonic motions during the whole simulation. The gas in this simulation is isothermal, so it is described by this equation of state:

$$p = \rho\,c_{\rm iso}^2$$

where $\rho$ is the gas density and $c_{\rm iso}$ is its isothermal sound speed.

The numerical simulation produces $101$ VTK files stored in:

- the **TURB_DRIVE_SUP_hr** folder: 

https://www.dropbox.com/scl/fo/aip4lo4983phsyd5tcn77/AD9ArKNSlPzlEpQxsnnRPWA/TURB_DRIVE_SUP_hr?rlkey=m4ragdal7lagwczhtkvyqj0ty&subfolder_nav_tracking=1&st=236scsha&dl=0

jointly with:

- a **units.out** file that contains the CGS normalisation values and the CGS value of the isothermal sound speed.
- a **vtk.out** file whose second column contains the times in code units.
- a **grid.out** file that contains information on the grid structure.

You can use VisIt to inspect the data. The written fields are: 

- density (rho)
- velocity_x (vx1)
- velocity_y (vx2)
- magnetic_field_x (Bx1)
- magnetic_field_y (Bx2)

**Reference paper:**
https://arxiv.org/pdf/0710.1359




## 2. (5 points) Shock analysis:

(c) Create a set of Python functions that isolate candidate shocked cells based on the following methods and sequentially prints the following shock candidate maps into a folder called **"output_s"** for all times:

- **Method 1:** Read the 2D velocity vector field. Compute the divergence of the velocity field and isolate the cells where there are convergent flows (i.e. where $\vec\nabla\cdot \vec v <\alpha$ with $\alpha\lesssim 0$). Cells with convergent flows are candidate shocked cells.

- **Method 2:** Read the 2D pressure field. Compute the gradient of the thermal pressure and isolate the cells with large pressure gradients (i.e. where $\frac{|\vec\nabla P|}{P}>\beta\max{\left(\frac{|\vec\nabla P|}{P}\right)}$ with $\beta \lesssim 0.1$). Such cells are candidate shocked cells.

(d) Make binary maps of the resulting candidate shock cells from both methods. Show sample maps at a fixed time (VTK file # 50) in this notebook.

## 3. (5 points) Fourier analysis:

(e) Create a set of Python functions that Fourier transforms the thermal pressure of any VTK file in 2D, applies high-pass and low-pass filters, smooths the pressure map using a 2D Gaussian, and sequentially prints the following figures into a folder called **"output_f"** for all times:

- the 2D Fourier image of the thermal pressure

- the high-pass filter map

- the low-pass filter map

- the Gaussian-blurred map

Show sample maps at a fixed time (VTK file # 50) in this notebook.

(f) Create a Python function that reads in the images you wrote, and returns movies showing the time evolution of the magnetic pressure gradient maps computed in (a), the high-pass filter maps in (e), and the flow circulation in (b).

## 4. (5 points) Interpretation:

(g) Based on your analyses above, briefly answer the following questions (**in your own words**): 

- What information do the velocity curl and magnetic pressure gradient provide about the flow in section 1?

- Does the flow circulation reach steady state in section 1?

- Do you find the same shock candidates on the maps obtained by Methods 1 and 2 in section 2?
  
- What do the high-pass and low-pass filter maps show in section 3?
  
- Are the low-pass and Gaussing blurring results consistent with one another in section 3?

## 1. (5 points) Numerical calculus:

(a) Create a set of Python functions that reads in the simulation data, normalises the data fields to CGS units (using units.out), returns all the data arrays and the mesh, and sequentially prints the following figures into a folder called **"output_n"** for all times:

- Thermal pressure, $p = \rho\,c_{\rm iso}^2$





In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pyvista as pv
import gc
ubication_principal=os.getcwd()

def read_file(name):
    file=ubication_principal+'/TURB_DRIVE_SUP_hr/'
    data=pd.DataFrame(pd.read_csv(file+name))
    rho=data.loc[0,'normalisation']
    v=data.loc[1,'normalisation']
    l=data.loc[2,'normalisation']
    c=data.loc[3,'normalisation']
    for i in range(4):
        print('************************')
        print(f'the {data.loc[i,'variable']} = {data.loc[i,'normalisation']} {data.loc[i,'units']}')
    return rho,v,l,c

    
rho,v0,l0,c_iso=read_file('units.out')


************************
the rho_0 = 1.66e-24 g/cm^3
************************
the v_0 = 1000000.0 cm/s
************************
the L_0 = 3.086e+18 cm
************************
the c_iso = 1000000.0 cm/s


In [2]:
# Normalize the pressure
p_norm=rho*(c_iso**2)
print(f'the presure is {p_norm} in g/(cm*s^2)')

the presure is 1.66e-12 in g/(cm*s^2)


In [3]:
## for read the vtk.out for time
def read_time(name):
    file=ubication_principal+'/TURB_DRIVE_SUP_hr/'
    data=pd.DataFrame(pd.read_csv(file+name,sep=' ',header=None))
    time=np.array(data[2])
    return time
# don't forgate that we have to normalize with velocity and length
t0=read_time('vtk.out')*(l0/v0)

    

In [4]:
#calculus of normalization for fiel magnetic
B=v0*np.sqrt(4*np.pi*rho)
B

np.float64(4.5672940807261605e-06)

In [5]:

def read_vtk(name):
    file=ubication_principal+'/TURB_DRIVE_SUP_hr/'
    data=pv.read(file+name)
    local_rho=data.get_array('rho')*rho
    vx=data.get_array('vx1')*v0
    vy=data.get_array('vx2')*v0
    bx=data.get_array('Bx1')*B
    by=data.get_array('Bx2')*B
    time=t0[int(name[5:9])]
    ##3 for create the meshure 
    data.x=data.x*l0
    data.y=data.y*l0
    data.z=data.z*l0
    dic={'rho':local_rho,'vx':vx,'vy':vy,'bx':bx,'by':by,'time':time}
    return data,dic


################ for graphics the data###########3
# chance the dimension for simulate

# adapt to in 2d the  variable

In [6]:

def simulation_pressure(name):
    
    data1,dics=read_vtk(name)
    x=np.linspace(data1.bounds[0],data1.bounds[1],data1.dimensions[0]-1)
    y=np.linspace(data1.bounds[2],data1.bounds[3],data1.dimensions[1]-1)
    x2d,y2d=np.meshgrid(x,y)
    #### variable
    press=(dics['rho'])*(((dics['vx'])**2)+((dics['vy'])**2))
    press2d=press.reshape(data1.dimensions[1]-1,data1.dimensions[0]-1)
    #for make the image
    fig,ax=plt.subplots(figsize=(10,10))
    mesh=ax.pcolor(x2d,y2d,press2d,cmap='magma')
    plt.colorbar(mesh,ax=ax,label='press')
    ax.set_title(f'file: {name}')
    ax.set_xlabel(f't(s)={dics['time']:.3e}')
    plt.savefig(name[:-4]+'.jpg')
    plt.close(fig)
    try:
        del data1
        del dics
        del x
        del y
        del x2d
        del y2d
        del press
        del press2d
    except NameError:
        pass

    gc.collect()

    




In [7]:
os.makedirs('output_n',exist_ok=True)
os.chdir('output_n')
os.makedirs('pressure',exist_ok=True)
os.chdir('pressure')

for i in range(len(t0)):
    name_n=f'data.{i:04d}.vtk'
    simulation_pressure(name_n)


os.chdir('..')
os.chdir('..')



In [ ]:
### for gif simulation 
from IPython import display
import glob
from PIL import Image
def gif_simulation(folder,name_gif):
    os.chdir('output_n')
    
    img_de=f'./{folder}/data.****.jpg'
    img_out=name_gif
    img=(Image.open(f) for f in sorted(glob.glob(img_de)))
    im=next(img)
    im.save(fp=img_out,format='GIF',append_images=img,save_all=True,duration=100,loop=0)
    display.Image(open(f'{name_gif}','rb').read())
    
    os.chdir('..')

gif_simulation('pressure','animation_pressure.gif')


In [17]:
def gif_simulation2(folder, name_gif):
    
    # 1. Guarda la ruta completa donde se guardará el GIF
    # La ruta es relativa a donde estás AHORA, no después de os.chdir
    ruta_gif = os.path.join(os.getcwd(), 'output_n', name_gif)
    
    os.chdir('output_n') # Entra en la carpeta
    
    img_de = f'./{folder}/data.****.jpg'
    img_out = name_gif

    img = Image.open(f for f in sorted(glob.glob(img_de)))
    im = next(img)
    
    # 2. Guarda el GIF usando la ruta COMPLETA para mayor seguridad
    im.save(fp=ruta_gif, format='GIF', append_images=img, save_all=True, duration=100, loop=0)

    os.chdir('..') # Sal de la carpeta
    
    # 3. VISUALIZACIÓN CORREGIDA: Usa display.Image con el nombre del archivo
    # Pasas la ruta del archivo, no los bytes.
    display.Image(filename=ruta_gif) # <-- ¡La corrección clave!

# Llama a la función (la visualización aparecerá en la celda)
gif_simulation2('pressure', 'animation_pressure2.gif')

AttributeError: 'generator' object has no attribute 'read'

os.getcwd()

- Velocity curl (vorticity) magnitude, $|\vec\nabla\times \vec v|$

- Magnetic pressure gradient, $\vec{\nabla}P_B$ where $P_B=\frac{B^2}{8\pi}$ is the magnetic pressure.

Add time-stamps in CGS units to the above maps (using information from vkt.out). Show sample maps at a fixed time (VTK file # 50) in this notebook.

(b) Create a set of Python functions that loops over all the simulation VTK files, computes the flow circulation, $\Gamma = \iint_S \vec\nabla\times \vec v \cdot \mathrm{d}\mathbf{S}$, and returns:

- a CSV file with 2 columns (time and flow circulation).

- a figure of the flow circulation versus time. Show the resulting figure in this notebook too.